# Gathering Data

In [1]:
from functions import *
from scivol import *
import nibabel as nib
import ipyvolume as ipv

In [2]:
proj_root = parent_directory()
print(proj_root)
mri_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T2w.nii.gz")
output_folder = os.path.join(proj_root, "output/")
mask_seq_path = "/Users/joachimpfefferkorn/repos/neurovolume/output/slice_images_mask"

/Users/joachimpfefferkorn/repos/neurovolume


In [3]:
anat_img = nib.load(mri_input_filepath)
full_anat = create_volume(normalize_array(anat_img.get_fdata()))
brain_mask = build_bool_mask(mask_seq_path, full_anat)
brain_anat = create_masked_normalized_tensor(full_anat, brain_mask)
ext_anat = create_masked_normalized_tensor(full_anat, brain_mask, keep_when=False)
# don't love the inconsistent nesting of these volume creation functions 


bool masks built v1
creating masked normal tensor
creating masked normal tensor


# Quick Visual Check
Quick note on the following `ipv` functions:
They appear to be normalizing the geometry to be a perfect cube. I wonder if there is a way to apply the affine to this?

In [4]:
#ipv.quickvolshow(full_anat)

In [5]:
#ipv.quickvolshow(brain_anat)

In [6]:
#ipv.quickvolshow(ext_anat)

# Building Scivol

In [7]:
affine = anat_img.affine
print(type(affine))

<class 'numpy.ndarray'>


Initialize the scivol object

In [8]:
skullstrip = Scivol("skullstrip", anat_img.affine, 0.2)

Create and add the grids

In [9]:
skullstrip.add_grids([Grid("full_anat",[full_anat]),
Grid("brain_anat", [brain_anat]),
Grid("ext_anat", [ext_anat])])


writing full_anat to grid
writing brain_anat to grid
writing ext_anat to grid


In [10]:
skullstrip.save_scivol(output_folder)

saving skullstrip.scivol
writing skullstrip.scivol


TypeError: Object of type ndarray is not JSON serializable

We will now move to blender to parse the scivol file